# import packages

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# import packages
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

from datetime import timedelta
import scipy.stats
from sklearn.decomposition import PCA
from torch.utils.data import Dataset, DataLoader

import copy

from scipy.interpolate import splev, splrep, interp1d

import sklearn
from sklearn.neighbors import NearestNeighbors

from prettytable import PrettyTable

from torchdiffeq import odeint_adjoint as odeint_adjoint
from torchdiffeq import odeint

In [3]:
# import functions and NNs

from adaptive_select import create_adap_data_buffer
from interpolate_windows import interpolate_window, interpolate_window_adap
from LSTM_functions import fit_LSTM_grids, pred_LSTM_grids, train_LSTM
from RNN_functions import fit_RNN_grids, pred_RNN_grids, train_RNN
from RNN_ODE_functions import fit_non_adap_models_grids, pred_non_adap_models_grids, train_non_adap_models
from RNN_ODE_Adap_functions import fit_adap_models, pred_adap_models, train_adap_models
from NN import RNNODE, OutputNN, RNN, LSTM

In [4]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

In [5]:
# set parameters
n_latent = 128
n_hidden = 128
time_scale = 10

# Generate spiral data

In [6]:
class Lambda(nn.Module):
    
    def __init__(self, true_A, alpha=2, beta = 3, gamma = 0, const = 1.):
        super(Lambda, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.true_A = true_A
        self.const = const
        
    def orig_f(self, t, y):
        return torch.mm(y, self.true_A) * (1/torch.norm(y)**self.alpha) #torch.mm(y**3, self.true_A) * (1/torch.norm(y)**self.alpha) 
    
    def sec_order_deriv(self, t, y):
        return (3/torch.norm(y)**self.alpha*self.orig_f(t, y) @ torch.diag((y.reshape((2,)))**2) @ self.true_A
              - self.alpha/torch.norm(y)**2 * torch.dot(y.reshape((2,)), self.orig_f(t, y).reshape((2,))) * self.orig_f(t, y))   

    def forward(self, t, y):
        return (self.orig_f(t, y) * (1/torch.norm(self.orig_f(t, y))**self.beta) 
                * torch.norm(self.sec_order_deriv(t, y))**self.gamma)*self.const


In [4]:
# generate spirals
num_traj   = 500
num_points_dense = 400
num_points_dense_discard = 0

alpha = 2
beta = 0
gamma = 0
const = 1


orig_ts_dense = torch.linspace(0,40,num_points_dense+1)
# orig_ts_dense = torch.linspace(60., 160., num_points_dense+1)
orig_samps_dense = []

true_A_mean = torch.tensor([[-0.1, -1.5], [1.5, -0.1]])#torch.tensor([[-0.1, -2], [2, -0.1]])
true_A_std = torch.tensor([[0.02, 0.04], [0.04, 0.02]])
true_y0 = torch.tensor([[0.6, 0.3]])
true_A_list = []

t1 = time.time()
for i in range(1,num_traj+1):
    true_A = torch.tensor(np.float32(np.random.normal(true_A_mean, true_A_std)))
    true_A_list.append(true_A)
    
    with torch.no_grad():
        orig_samps_temp = odeint(Lambda(-true_A, alpha=alpha, beta=beta, gamma=gamma, const=const), true_y0, orig_ts_dense, method='dopri5')[num_points_dense_discard:,0,:].reshape(
            (num_points_dense-num_points_dense_discard+1,2))
    orig_samps_dense.append(orig_samps_temp)
    if i % 50 == 1:
        t3 = time.time()
        remain_time = (t3-t1) * (num_traj-i) / i
        print('generating the %3d-the spiral now, remaining time = %5.2f s' % (i, remain_time))
t2 = time.time()
print('generating spirals takes %.2f s' % (t2-t1))
    
orig_samps_dense = torch.stack(orig_samps_dense).reshape((num_traj, num_points_dense-num_points_dense_discard+1, 2))
orig_ts_dense = orig_ts_dense[num_points_dense_discard:]


generating the   1-the spiral now, remaining time = 263.48 s
generating the  51-the spiral now, remaining time = 282.36 s
generating the 101-the spiral now, remaining time = 251.14 s
generating the 151-the spiral now, remaining time = 216.07 s
generating the 201-the spiral now, remaining time = 191.26 s
generating the 251-the spiral now, remaining time = 157.35 s
generating the 301-the spiral now, remaining time = 122.57 s
generating the 351-the spiral now, remaining time = 89.91 s
generating the 401-the spiral now, remaining time = 59.62 s
generating the 451-the spiral now, remaining time = 29.64 s
generating spirals takes 301.89 s


In [9]:
num_points = 400
subsamp_ind = np.linspace(0, num_points_dense-num_points_dense_discard, num_points+1, dtype=int)
orig_samps = orig_samps_dense[:,subsamp_ind,:]
orig_ts = orig_ts_dense[subsamp_ind]

subsamp_ind.shape

(401,)

# Generate training and testing windows

In [10]:
num_fine_grid = 64
num_layers = 3
num_coarse_grid = num_fine_grid // 2**(num_layers-1)


In [11]:
seed_everything()
rng = np.random.default_rng() 

obs_dim = 2
num_train_windows = 500
num_rep = 50

In [12]:
def sample_windows(trajs_orig, ts_orig, window_length=num_fine_grid+1, num_trajs=250, num_window_per_traj=20,
                  ind_start=2, ind_end=200):
    train_windows0 = []
    train_ts0 = []
    train_start0 = []

    train_ind_list_candi = np.arange(ind_start,ind_end-window_length)
    for j in range(num_trajs):
        train_ind_list = np.sort(rng.choice(train_ind_list_candi, num_window_per_traj))
        train_start0.append(train_ind_list)
        for i in range(num_window_per_traj):
            start = train_ind_list[i]
            train_windows0.append(torch.from_numpy(np.float32(
                trajs_orig[j,start:start+window_length,:])))
            train_ts0.append(torch.from_numpy(np.float32(ts_orig[start:start+window_length])))        

    train_windows0 = torch.stack(train_windows0)
    train_ts0 = torch.stack(train_ts0)

    return train_windows0, train_ts0, train_start0

In [13]:
# generate windows

train_windows0_list, train_ts0_list, train_start0_list = [], [], []
test_windows0_list, test_ts0_list, test_start0_list = [], [], []

num_traj_train = 250
num_window_per_traj = 2

for rep in range(num_rep):
    train_windows0, train_ts0, train_start0 = sample_windows(
        orig_samps, orig_ts, window_length=num_fine_grid+1, num_trajs=num_traj_train, num_window_per_traj=num_window_per_traj,
          ind_start=2, ind_end=num_points)
#             print(train_windows0.shape)
    test_windows0, test_ts0, test_start0 = sample_windows(
        orig_samps[num_traj_train:], orig_ts, window_length=num_fine_grid+1, num_trajs=num_traj_train, num_window_per_traj=num_window_per_traj,
          ind_start=2, ind_end=num_points)
    train_windows0_list.append(train_windows0)
    train_ts0_list.append(train_ts0)
    train_start0_list.append(train_start0)
    test_windows0_list.append(test_windows0)
    test_ts0_list.append(test_ts0)
    test_start0_list.append(test_start0)


In [14]:
# customize dataset used in training models
class MyDataset(Dataset):
    def __init__(self, train_windows, train_ts):
        self.train_windows = train_windows
        self.train_ts = train_ts
        
    def __getitem__(self, index):
        window_temp = self.train_windows[index]
        window_ts_temp = self.train_ts[index]
        
        return window_temp, window_ts_temp

    def __len__(self):
        return self.train_windows.shape[0]
    
class MyDataset_adap(Dataset):
    def __init__(self, train_windows, train_ts, train_data_len):
        self.train_windows = train_windows
        self.train_ts = train_ts
        self.train_data_len = train_data_len
        
    def __getitem__(self, index):
        window_temp = self.train_windows[index]
        window_ts_temp = self.train_ts[index]
        window_temp_len = self.train_data_len[index]
        
        return window_temp, window_ts_temp, window_temp_len

    def __len__(self):
        return self.train_windows.shape[0]

# RNN_ODE models

In [ ]:
# RNN_ODE, number of grids >= 16


num_grids_non_adap_list1 = np.arange(16,68,4, dtype=int)+1#np.arange(24,102,6, dtype=int)+1#

odefunc_non_adap_list1, outputfunc_non_adap_list1 = [], []
fit_window_train_non_adap_list1, fit_window_test_non_adap_list1 = [], []
fit_L2_err_train_non_adap_list1, fit_L2_err_test_non_adap_list1 = [], []
fit_L1_err_train_non_adap_list1, fit_L1_err_test_non_adap_list1 = [], []

fit_L2_err_train_non_adap_mean_list1, fit_L2_err_test_non_adap_mean_list1 = [], []
fit_L2_err_train_non_adap_std_list1, fit_L2_err_test_non_adap_std_list1 = [], []


pred_window_train_non_adap_list1, pred_window_test_non_adap_list1 = [], []
pred_window_train_non_adap_list1_pre, pred_window_test_non_adap_list1_pre = [], []
pred_L2_err_train_non_adap_list1, pred_L2_err_test_non_adap_list1 = [], []
pred_L1_err_train_non_adap_list1, pred_L1_err_test_non_adap_list1 = [], []

pred_L2_err_train_non_adap_mean_list1, pred_L2_err_test_non_adap_mean_list1 = [], []
pred_L2_err_train_non_adap_std_list1, pred_L2_err_test_non_adap_std_list1 = [], []

    
batch_size = 64#128
verbose = 2
method = "naiveEuler"#"rk4"
buffer_start_steps = 2
n_iter = 40
rescale_const = 1

t1 = time.time()
index = np.arange(num_fine_grid+1)
for i, num_grids in enumerate(num_grids_non_adap_list1):
    print('\n----------------------------------------------------------------\n')
    print('\nnon-adaptive method, number of grids = %d\n' % num_grids)
    index1 = np.round(np.arange(num_grids-1) * num_fine_grid/(num_grids-1))
    index1 = np.concatenate(([index1[0]]*buffer_start_steps, index1, [num_fine_grid])) 

    print('train models:\n')

    odefunc1_list, outputfunc1_list = [], []
    fit_window_train1_list, fit_window_test1_list = [], []
    fit_L2_err_train1_list, fit_L2_err_test1_list = [], []
    fit_L1_err_train1_list, fit_L1_err_test1_list = [], []

    pred_window_train1_list, pred_window_test1_list = [], []
    pred_window_train1_list_pre, pred_window_test1_list_pre = [], []
    pred_L2_err_train1_list, pred_L2_err_test1_list = [], []
    pred_L1_err_train1_list, pred_L1_err_test1_list = [], []

    rep = 0
    while rep <= num_rep-1:
        
        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]
        
        train_ts1 = torch.tensor(np.linspace(train_ts0[:,0], train_ts0[:,-1], num_grids).T)
        train_ts1 = torch.concat((train_ts1[:,0].reshape(train_ts0.shape[0],1).repeat(1,buffer_start_steps), 
                                train_ts1),axis=1)
        train_windows1 = interpolate_window(train_windows0, train_ts0, train_ts1, 'cubic')
        
        if buffer_start_steps > 0:
            deltat = train_ts1[0][-1] - train_ts1[0][-2]
            train_ts1[:,:buffer_start_steps] = (train_ts1[:,:buffer_start_steps] + 
                                                torch.arange(-deltat*buffer_start_steps, 0, deltat))
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     

        train_loader1 = torch.utils.data.DataLoader(
            MyDataset(train_windows=train_windows1[train_index,:,:], train_ts=train_ts1[train_index,:]), 
            shuffle=True, batch_size=batch_size,) # rescale the data

        input_steps1 = train_windows1.shape[1]  

        print('\n------the %d-th replica:--------' % (rep+1))
        flag, odefunc1, outputfunc1 = train_non_adap_models(train_loader1, input_steps1, valid_windows0, valid_ts0, num_grids,  
                                                            verbose, n_iter=n_iter, method=method, thres1=6.5e2, thres2=6.5e2, weight=(1,0),
                                                            obs_dim=obs_dim, n_hidden=n_hidden, n_latent=n_latent, num_train_windows=500, 
                                                            time_scale=time_scale, buffer_start_steps=buffer_start_steps)
    
        if not flag:
            continue
        else:
            rep += 1
        
        print('\nfitting error of training data:')
        fit_window_train1, fit_L2_err_train1, fit_L1_err_train1 = fit_non_adap_models_grids(
            odefunc1, outputfunc1, train_windows0, train_ts0, num_grids, method=method, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)
        print('\nfitting error of testing data:')
        fit_window_test1, fit_L2_err_test1, fit_L1_err_test1 = fit_non_adap_models_grids(
            odefunc1, outputfunc1, test_windows0, test_ts0, num_grids, method=method, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)

        print('\npredicting error of training data:')
        pred_window_train1_pre, pred_window_train1, pred_L2_err_train1, pred_L1_err_train1 = pred_non_adap_models_grids(
            odefunc1, outputfunc1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)
        print('\npredicting error of testing data:')
        pred_window_test1_pre, pred_window_test1, pred_L2_err_test1, pred_L1_err_test1 = pred_non_adap_models_grids(
            odefunc1, outputfunc1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)

        odefunc1_list.append(copy.deepcopy(odefunc1))
        outputfunc1_list.append(copy.deepcopy(outputfunc1))
        fit_L2_err_train1_list.append(fit_L2_err_train1)
        fit_L2_err_test1_list.append(fit_L2_err_test1)
        fit_L1_err_train1_list.append(fit_L1_err_train1)
        fit_L1_err_test1_list.append(fit_L1_err_test1)
        fit_window_train1_list.append(fit_window_train1)
        fit_window_test1_list.append(fit_window_test1)

        pred_L2_err_train1_list.append(pred_L2_err_train1)
        pred_L2_err_test1_list.append(pred_L2_err_test1)
        pred_L1_err_train1_list.append(pred_L1_err_train1)
        pred_L1_err_test1_list.append(pred_L1_err_test1)
        pred_window_train1_list.append(pred_window_train1)
        pred_window_test1_list.append(pred_window_test1)
        pred_window_train1_list_pre.append(pred_window_train1_pre)
        pred_window_test1_list_pre.append(pred_window_test1_pre)

    odefunc_non_adap_list1.append(odefunc1_list)
    outputfunc_non_adap_list1.append(outputfunc1_list)
    fit_window_train_non_adap_list1.append(fit_window_train1_list)
    fit_window_test_non_adap_list1.append(fit_window_test1_list)
    fit_L2_err_train_non_adap_list1.append(fit_L2_err_train1_list)
    fit_L2_err_test_non_adap_list1.append(fit_L2_err_test1_list)
    fit_L1_err_train_non_adap_list1.append(fit_L1_err_train1_list)
    fit_L1_err_test_non_adap_list1.append(fit_L1_err_test1_list)    
    
    fit_L2_err_train_ave = np.mean(torch.stack(fit_L2_err_train1_list).numpy(),axis=(1,))
    fit_L2_err_test_ave = np.mean(torch.stack(fit_L2_err_test1_list).numpy(),axis=(1,))
    fit_L2_err_train_non_adap_mean_list1.append(np.mean(fit_L2_err_train_ave))
    fit_L2_err_train_non_adap_std_list1.append(np.std(fit_L2_err_train_ave))
    fit_L2_err_test_non_adap_mean_list1.append(np.mean(fit_L2_err_test_ave))
    fit_L2_err_test_non_adap_std_list1.append(np.std(fit_L2_err_test_ave))
    
    print('\nL2 fitting error of training data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave),
                                                                         np.std(fit_L2_err_train_ave)))
    print('\nL2 fitting error of testing data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_test_ave),
                                                                         np.std(fit_L2_err_test_ave)))
    
    pred_window_train_non_adap_list1.append(pred_window_train1_list)
    pred_window_test_non_adap_list1.append(pred_window_test1_list)
    pred_window_train_non_adap_list1_pre.append(pred_window_train1_list_pre)
    pred_window_test_non_adap_list1_pre.append(pred_window_test1_list_pre)
    pred_L2_err_train_non_adap_list1.append(pred_L2_err_train1_list)
    pred_L2_err_test_non_adap_list1.append(pred_L2_err_test1_list)
    pred_L1_err_train_non_adap_list1.append(pred_L1_err_train1_list)
    pred_L1_err_test_non_adap_list1.append(pred_L1_err_test1_list)    
    
    pred_L2_err_train_ave = np.mean(torch.stack(pred_L2_err_train1_list).numpy(),axis=(1,))
    pred_L2_err_test_ave = np.mean(torch.stack(pred_L2_err_test1_list).numpy(),axis=(1,))
    pred_L2_err_train_non_adap_mean_list1.append(np.mean(pred_L2_err_train_ave))
    pred_L2_err_train_non_adap_std_list1.append(np.std(pred_L2_err_train_ave))
    pred_L2_err_test_non_adap_mean_list1.append(np.mean(pred_L2_err_test_ave))
    pred_L2_err_test_non_adap_std_list1.append(np.std(pred_L2_err_test_ave))
    
    print('\nL2 predicting error of training data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_train_ave),
                                                                         np.std(pred_L2_err_train_ave)))
    print('\nL2 predicting error of testing data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave),
                                                                         np.std(pred_L2_err_test_ave)))
    
t2 = time.time()
print('training model takes %.2f s' % (t2-t1) )
    

In [ ]:
# non-adaptive method, interpolate to get the regular time steps (validation data), smaller number of grids


num_grids_non_adap_list2 = np.array([6,8,12])+1#np.arange(24,102,6, dtype=int)+1#

odefunc_non_adap_list2, outputfunc_non_adap_list2 = [], []
fit_window_train_non_adap_list2, fit_window_test_non_adap_list2 = [], []
fit_L2_err_train_non_adap_list2, fit_L2_err_test_non_adap_list2 = [], []
fit_L1_err_train_non_adap_list2, fit_L1_err_test_non_adap_list2 = [], []

fit_L2_err_train_non_adap_mean_list2, fit_L2_err_test_non_adap_mean_list2 = [], []
fit_L2_err_train_non_adap_std_list2, fit_L2_err_test_non_adap_std_list2 = [], []


pred_window_train_non_adap_list2, pred_window_test_non_adap_list2 = [], []
pred_window_train_non_adap_list2_pre, pred_window_test_non_adap_list2_pre = [], []
pred_L2_err_train_non_adap_list2, pred_L2_err_test_non_adap_list2 = [], []
pred_L1_err_train_non_adap_list2, pred_L1_err_test_non_adap_list2 = [], []

pred_L2_err_train_non_adap_mean_list2, pred_L2_err_test_non_adap_mean_list2 = [], []
pred_L2_err_train_non_adap_std_list2, pred_L2_err_test_non_adap_std_list2 = [], []

    

batch_size = 64#128
verbose = 2
method = "naiveEuler"#"rk4"
buffer_start_steps = 2

t1 = time.time()
index = np.arange(num_fine_grid+1)
for i, num_grids in enumerate(num_grids_non_adap_list2):
    print('\n----------------------------------------------------------------\n')
    print('\nnon-adaptive method, number of grids = %d\n' % num_grids)
    index1 = np.round(np.arange(num_grids-1) * num_fine_grid/(num_grids-1))
    index1 = np.concatenate(([index1[0]]*buffer_start_steps, index1, [num_fine_grid])) 

    print('train models:\n')

    odefunc1_list, outputfunc1_list = [], []
    fit_window_train1_list, fit_window_test1_list = [], []
    fit_L2_err_train1_list, fit_L2_err_test1_list = [], []
    fit_L1_err_train1_list, fit_L1_err_test1_list = [], []

    pred_window_train1_list, pred_window_test1_list = [], []
    pred_window_train1_list_pre, pred_window_test1_list_pre = [], []
    pred_L2_err_train1_list, pred_L2_err_test1_list = [], []
    pred_L1_err_train1_list, pred_L1_err_test1_list = [], []

    rep = 0
    while rep <= num_rep-1:
        
        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]
        
        train_ts1 = torch.tensor(np.linspace(train_ts0[:,0], train_ts0[:,-1], num_grids).T)
        train_ts1 = torch.concat((train_ts1[:,0].reshape(train_ts0.shape[0],1).repeat(1,buffer_start_steps), 
                                train_ts1),axis=1)
        train_windows1 = interpolate_window(train_windows0, train_ts0, train_ts1, 'cubic')
        
        if buffer_start_steps > 0:
            deltat = train_ts1[0][-1] - train_ts1[0][-2]
            train_ts1[:,:buffer_start_steps] = (train_ts1[:,:buffer_start_steps] + 
                                                torch.arange(-deltat*buffer_start_steps, 0, deltat))
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     

        train_loader1 = torch.utils.data.DataLoader(
            MyDataset(train_windows=train_windows1[train_index,:,:], train_ts=train_ts1[train_index,:]), 
            shuffle=True, batch_size=batch_size,) # rescale the data

        input_steps1 = train_windows1.shape[1]  

        print('\n------the %d-th replica:--------' % (rep+1))
        flag, odefunc1, outputfunc1 = train_non_adap_models(train_loader1, input_steps1, valid_windows0, valid_ts0, num_grids,  
                                                            verbose, n_iter=n_iter, method=method, thres1=6.5e2, thres2=6.5e2, weight=(1,0),
                                                            obs_dim=obs_dim, n_hidden=n_hidden, n_latent=n_latent, num_train_windows=500, 
                                                            time_scale=time_scale, buffer_start_steps=buffer_start_steps)
        if not flag:
            continue
        else:
            rep += 1
        
        print('\nfitting error of training data:')
        fit_window_train1, fit_L2_err_train1, fit_L1_err_train1 = fit_non_adap_models_grids(
            odefunc1, outputfunc1, train_windows0, train_ts0, num_grids, method=method, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)
        print('\nfitting error of testing data:')
        fit_window_test1, fit_L2_err_test1, fit_L1_err_test1 = fit_non_adap_models_grids(
            odefunc1, outputfunc1, test_windows0, test_ts0, num_grids, method=method, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)

        print('\npredicting error of training data:')
        pred_window_train1_pre, pred_window_train1, pred_L2_err_train1, pred_L1_err_train1 = pred_non_adap_models_grids(
            odefunc1, outputfunc1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)
        print('\npredicting error of testing data:')
        pred_window_test1_pre, pred_window_test1, pred_L2_err_test1, pred_L1_err_test1 = pred_non_adap_models_grids(
            odefunc1, outputfunc1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim, rescale_const=rescale_const, time_scale=time_scale)

        odefunc1_list.append(copy.deepcopy(odefunc1))
        outputfunc1_list.append(copy.deepcopy(outputfunc1))
        fit_L2_err_train1_list.append(fit_L2_err_train1)
        fit_L2_err_test1_list.append(fit_L2_err_test1)
        fit_L1_err_train1_list.append(fit_L1_err_train1)
        fit_L1_err_test1_list.append(fit_L1_err_test1)
        fit_window_train1_list.append(fit_window_train1)
        fit_window_test1_list.append(fit_window_test1)

        pred_L2_err_train1_list.append(pred_L2_err_train1)
        pred_L2_err_test1_list.append(pred_L2_err_test1)
        pred_L1_err_train1_list.append(pred_L1_err_train1)
        pred_L1_err_test1_list.append(pred_L1_err_test1)
        pred_window_train1_list.append(pred_window_train1)
        pred_window_test1_list.append(pred_window_test1)
        pred_window_train1_list_pre.append(pred_window_train1_pre)
        pred_window_test1_list_pre.append(pred_window_test1_pre)

    odefunc_non_adap_list2.append(odefunc1_list)
    outputfunc_non_adap_list2.append(outputfunc1_list)
    fit_window_train_non_adap_list2.append(fit_window_train1_list)
    fit_window_test_non_adap_list2.append(fit_window_test1_list)
    fit_L2_err_train_non_adap_list2.append(fit_L2_err_train1_list)
    fit_L2_err_test_non_adap_list2.append(fit_L2_err_test1_list)
    fit_L1_err_train_non_adap_list2.append(fit_L1_err_train1_list)
    fit_L1_err_test_non_adap_list2.append(fit_L1_err_test1_list)    
    
    fit_L2_err_train_ave = np.mean(torch.stack(fit_L2_err_train1_list).numpy(),axis=(1,))
    fit_L2_err_test_ave = np.mean(torch.stack(fit_L2_err_test1_list).numpy(),axis=(1,))
    fit_L2_err_train_non_adap_mean_list2.append(np.mean(fit_L2_err_train_ave))
    fit_L2_err_train_non_adap_std_list2.append(np.std(fit_L2_err_train_ave))
    fit_L2_err_test_non_adap_mean_list2.append(np.mean(fit_L2_err_test_ave))
    fit_L2_err_test_non_adap_std_list2.append(np.std(fit_L2_err_test_ave))
    
    print('\nL2 fitting error of training data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave),
                                                                         np.std(fit_L2_err_train_ave)))
    print('\nL2 fitting error of testing data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_test_ave),
                                                                         np.std(fit_L2_err_test_ave)))
    
    pred_window_train_non_adap_list2.append(pred_window_train1_list)
    pred_window_test_non_adap_list2.append(pred_window_test1_list)
    pred_window_train_non_adap_list2_pre.append(pred_window_train1_list_pre)
    pred_window_test_non_adap_list2_pre.append(pred_window_test1_list_pre)
    pred_L2_err_train_non_adap_list2.append(pred_L2_err_train1_list)
    pred_L2_err_test_non_adap_list2.append(pred_L2_err_test1_list)
    pred_L1_err_train_non_adap_list2.append(pred_L1_err_train1_list)
    pred_L1_err_test_non_adap_list2.append(pred_L1_err_test1_list)    
    
    pred_L2_err_train_ave = np.mean(torch.stack(pred_L2_err_train1_list).numpy(),axis=(1,))
    pred_L2_err_test_ave = np.mean(torch.stack(pred_L2_err_test1_list).numpy(),axis=(1,))
    pred_L2_err_train_non_adap_mean_list2.append(np.mean(pred_L2_err_train_ave))
    pred_L2_err_train_non_adap_std_list2.append(np.std(pred_L2_err_train_ave))
    pred_L2_err_test_non_adap_mean_list2.append(np.mean(pred_L2_err_test_ave))
    pred_L2_err_test_non_adap_std_list2.append(np.std(pred_L2_err_test_ave))
    
    print('\nL2 predicting error of training data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_train_ave),
                                                                         np.std(pred_L2_err_train_ave)))
    print('\nL2 predicting error of testing data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave),
                                                                         np.std(pred_L2_err_test_ave)))
    
t2 = time.time()
print('training model takes %.2f s' % (t2-t1) )
    

In [ ]:
# compute errors

pred_L2_err_test_non_adap_list_mean_mat1 = np.zeros((num_rep, len(num_grids_non_adap_list1)))

for i in range(len(num_grids_non_adap_list1)):
    pred_L2_err_test_non_adap_list_mean_mat1[:,i] = torch.stack(pred_L2_err_test_non_adap_list1[i]).mean(axis=(1,)).numpy()

    
pred_L2_err_test_non_adap_list_mean_mat2 = np.zeros((num_rep, len(num_grids_non_adap_list2)))

for i in range(len(num_grids_non_adap_list2)):
    pred_L2_err_test_non_adap_list_mean_mat2[:,i] = torch.stack(pred_L2_err_test_non_adap_list2[i]).mean(axis=(1,)).numpy()
    


# RNN_ODE_Adap models

In [ ]:
# RNN_ODE_Adap, number of grids >= 16

thres_list1 = ([0, 0.565, 0.615, 0.66, 0.73, 0.81, 0.91, 1.09] + 
              [0.84, 0.91, 1.02, 1.18, 3])

rescale_const = 1
thres_list1.reverse()


verbose = 2
n_iter = 50

num_grids_adap_list1 = []

odefunc_adap_list1, outputfunc_adap_list1 = [], []

# fitting errors
fit_window_train_adap_list1_cubic, fit_window_test_adap_list1_cubic = [], []
fit_L2_err_train_adap_list1_cubic, fit_L2_err_test_adap_list1_cubic = [], []
fit_L1_err_train_adap_list1_cubic, fit_L1_err_test_adap_list1_cubic = [], []

fit_L2_err_train_adap_mean_list1_cubic, fit_L2_err_test_adap_mean_list1_cubic = [], []
fit_L2_err_train_adap_std_list1_cubic, fit_L2_err_test_adap_std_list1_cubic = [], []


# predicting errors
pred_window_test_adap_list1_cubic = []
pred_window_test_adap_list1_cubic_pre = []
pred_L2_err_test_adap_list1_cubic = []
pred_L1_err_test_adap_list1_cubic = []

pred_L2_err_test_adap_mean_list1_cubic = []
pred_L2_err_test_adap_std_list1_cubic = []


for i, thres in enumerate(thres_list1):
        
#     if i <= 1:
#         num_fine_adap = 64
#         num_layers_adap = 4        
#     elif i <= 4:
#         num_fine_adap = 64
#         num_layers_adap = 3
#     else:
#         num_fine_adap = 64
#         num_layers_adap = 2
    if i <= 4:
        num_fine_adap = 64
        num_layers_adap = 3       
    else:
        num_fine_adap = 64
        num_layers_adap = 2
        
    print('\n----------------------------------------------------------------\n')

    odefunc1_list1, outputfunc1_list1 = [], []
    
    fit_window_train1_list1_cubic, fit_window_test1_list1_cubic = [], []
    fit_L2_err_train1_list1_cubic, fit_L2_err_test1_list1_cubic = [], []
    fit_L1_err_train1_list1_cubic, fit_L1_err_test1_list1_cubic = [], []
    fit_window_train1_list1_linear, fit_window_test1_list1_linear = [], []
    fit_L2_err_train1_list1_linear, fit_L2_err_test1_list1_linear = [], []
    fit_L1_err_train1_list1_linear, fit_L1_err_test1_list1_linear = [], []

    pred_window_test1_list1_cubic = []
    pred_window_test1_list1_cubic_pre = []
    pred_L2_err_test1_list1_cubic = []
    pred_L1_err_test1_list1_cubic = []
    pred_window_test1_list1_linear = []
    pred_window_test1_list1_linear_pre = []
    pred_L2_err_test1_list1_linear = []
    pred_L1_err_test1_list1_linear = []
    
    num_grids_adap_list1_temp = []

    rep = 0
    while rep <= num_rep-1:
        
        print('\n------the %d-th replica:--------' % (rep+1))

        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]

        train_windows0_adap_buffer, train_ts0_adap_buffer, train_len0_adap_buffer, train_windows_adap_len = create_adap_data_buffer(
            train_windows0, train_ts0, thres, num_fine_adap, num_layers_adap)
        test_windows0_adap_buffer, test_ts0_adap_buffer, test_len0_adap_buffer, _ = create_adap_data_buffer(
            test_windows0, test_ts0, thres, num_fine_adap, num_layers_adap)
        # train_windows, 1st half
        train_windows0_adap_buffer_trunc, train_ts0_adap_buffer_trunc, _, _, train_time_index = create_adap_data_buffer(
            train_windows0[:,:(train_ts0.shape[1]+1)//2,:], train_ts0[:,:(train_ts0.shape[1]+1)//2], thres, num_fine_adap//2, num_layers_adap, return_index=1)
        # train_windows, 2nd half
        _, train_ts0_adap_buffer_trunc2, train_len0_adap_buffer_trunc2, _ = create_adap_data_buffer(
            train_windows0[:,-(train_ts0.shape[1]+1)//2:,:], train_ts0[:,-(train_ts0.shape[1]+1)//2:], thres, num_fine_adap//2, num_layers_adap, buffer_start_steps=0)
        # test_windows, 1st half
        test_windows0_adap_buffer_trunc, test_ts0_adap_buffer_trunc, _, _, test_time_index = create_adap_data_buffer(
            test_windows0[:,:(test_ts0.shape[1]+1)//2,:], test_ts0[:,:(test_ts0.shape[1]+1)//2], thres, num_fine_adap//2, num_layers_adap, return_index=1)
        # test_windows, 2nd half
        _, test_ts0_adap_buffer_trunc2, test_len0_adap_buffer_trunc2, _ = create_adap_data_buffer(
            test_windows0[:,-(test_ts0.shape[1]+1)//2:,:], test_ts0[:,-(test_ts0.shape[1]+1)//2:], thres, num_fine_adap//2, num_layers_adap, buffer_start_steps=0)

        num_grids_adap_list1_temp.append(train_windows_adap_len)
        
#         print(train_windows0[:,:(train_ts0.shape[1]+1)//2,:].shape)

        print('\nadaptive method, thres = %.2f, number of grids = %.1f (%.2e)\n' % (thres, train_windows_adap_len, train_windows_adap_len))
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        valid_windows0_adap_buffer = train_windows0_adap_buffer[valid_index,:,:]
        valid_ts0_adap_buffer = train_ts0_adap_buffer[valid_index,:]
        valid_windows0_adap_buffer_trunc = train_windows0_adap_buffer_trunc[valid_index,:,:]
        valid_ts0_adap_buffer_trunc = train_ts0_adap_buffer_trunc[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     
        train_loader_adap1_buffer = torch.utils.data.DataLoader(
            MyDataset_adap(train_windows=train_windows0_adap_buffer[train_index], train_ts=train_ts0_adap_buffer[train_index], 
                            train_data_len=np.array(train_len0_adap_buffer)[train_index]), 
            shuffle=True, batch_size=batch_size,) 
        input_steps_adap1_buffer = max(train_len0_adap_buffer)   
        
        print('train models:\n')
        
        if i <= 1:
            valid_ts_adap_trunc2, valid_len_adap_trunc2 = valid_ts0[:, (valid_ts0.shape[1]-1)//2::1], [(valid_ts0.shape[1]-1)//2+1]*test_ts0.shape[0]
        elif i <= 4:
            valid_ts_adap_trunc2, valid_len_adap_trunc2 = valid_ts0[:, (valid_ts0.shape[1]-1)//2::1], [(valid_ts0.shape[1]-1)//2+1]*test_ts0.shape[0]
        else:
            valid_ts_adap_trunc2, valid_len_adap_trunc2 = valid_ts0[:, (valid_ts0.shape[1]-1)//2::1], [(valid_ts0.shape[1]-1)//2+1]*test_ts0.shape[0]

        flag, odefunc_adap1_buffer, outputfunc_adap1_buffer = train_adap_models(
            train_loader_adap1_buffer, input_steps_adap1_buffer, verbose, 
            valid_window=valid_windows0, valid_ts=valid_ts0, valid_window_adap=valid_windows0_adap_buffer, 
            valid_ts_adap=valid_ts0_adap_buffer, valid_window_adap_trunc=valid_windows0_adap_buffer_trunc, 
            valid_ts_adap_trunc=valid_ts0_adap_buffer_trunc, buffer_start_steps=buffer_start_steps, 
            weight=(1,0), n_iter=n_iter, thres1=1.5e3, valid_len=np.array(train_len0_adap_buffer)[valid_index], 
            valid_ts_adap_trunc2=valid_ts_adap_trunc2, valid_len_adap_trunc2=valid_len_adap_trunc2,
            pred_len=(train_ts0.shape[1]-1)//2, rescale_const=rescale_const,
            n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )    
        
        if not flag:
            continue
        else:
            rep += 1
            
        odefunc1_list1.append(copy.deepcopy(odefunc_adap1_buffer))
        outputfunc1_list1.append(copy.deepcopy(outputfunc_adap1_buffer))
        
        if i <= 1:
            test_ts_adap_trunc2, test_len_adap_trunc2 = test_ts0[:, (test_ts0.shape[1]-1)//2::1], [(test_ts0.shape[1]-1)//2+1]*test_ts0.shape[0]
        elif i <= 4:
            test_ts_adap_trunc2, test_len_adap_trunc2 = test_ts0[:, (test_ts0.shape[1]-1)//2::1], [(test_ts0.shape[1]-1)//2+1]*test_ts0.shape[0]
        else:
            test_ts_adap_trunc2, test_len_adap_trunc2 = test_ts0[:, (test_ts0.shape[1]-1)//2::1], [(test_ts0.shape[1]-1)//2+1]*test_ts0.shape[0]
        print(test_ts_adap_trunc2.shape)
        interp_kind = 'cubic'
        print('\nfitting error of training data (cubic):')
        fit_window_adap_train1, fit_L2_err_adap_train1, fit_L1_err_adap_train1 = fit_adap_models(
            odefunc_adap1_buffer, outputfunc_adap1_buffer, train_windows0, train_ts0, 
            train_windows0_adap_buffer, train_ts0_adap_buffer, train_len0_adap_buffer, buffer_start_steps,
            interp_kind=interp_kind, rescale_const=rescale_const, n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )   
        print('fitting error of testing data (cubic):')
        fit_window_adap_test1, fit_L2_err_adap_test1, fit_L1_err_adap_test1 = fit_adap_models(
            odefunc_adap1_buffer, outputfunc_adap1_buffer, test_windows0, test_ts0, 
            test_windows0_adap_buffer, test_ts0_adap_buffer, test_len0_adap_buffer, buffer_start_steps,
            interp_kind=interp_kind, rescale_const=rescale_const, n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )    
        fit_L2_err_train1_list1_cubic.append(fit_L2_err_adap_train1)
        fit_L2_err_test1_list1_cubic.append(fit_L2_err_adap_test1)
        fit_L1_err_train1_list1_cubic.append(fit_L1_err_adap_train1)
        fit_L1_err_test1_list1_cubic.append(fit_L1_err_adap_test1)
        fit_window_train1_list1_cubic.append(fit_window_adap_train1)
        fit_window_test1_list1_cubic.append(fit_window_adap_test1)

        print('predicting error of testing data (cubic):')
        pred_window_adap_test1_pre, pred_window_adap_test1, pred_L2_err_adap_test1, pred_L1_err_adap_test1 = pred_adap_models(
            odefunc_adap1_buffer, outputfunc_adap1_buffer, test_windows0, test_ts0, 
            test_windows0_adap_buffer_trunc, test_ts0_adap_buffer_trunc, rescale_const=rescale_const,
            ts_adap_buffer_trunc2=test_ts_adap_trunc2, len_adap_buffer_trunc2=test_len_adap_trunc2, 
            pred_len=(train_ts0.shape[1]-1)//2, buffer_start_steps=buffer_start_steps,interp_kind=interp_kind,
            n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )     

        pred_L2_err_test1_list1_cubic.append(pred_L2_err_adap_test1)
        pred_L1_err_test1_list1_cubic.append(pred_L1_err_adap_test1)
        pred_window_test1_list1_cubic.append(pred_window_adap_test1)
        pred_window_test1_list1_cubic_pre.append(pred_window_adap_test1_pre)
        
        
        
    odefunc_adap_list1.append(odefunc1_list1)
    outputfunc_adap_list1.append(outputfunc1_list1)
    num_grids_adap_list1.append(np.mean(num_grids_adap_list1_temp))
    
    fit_window_train_adap_list1_cubic.append(fit_window_train1_list1_cubic)
    fit_window_test_adap_list1_cubic.append(fit_window_test1_list1_cubic)
    fit_L2_err_train_adap_list1_cubic.append(fit_L2_err_train1_list1_cubic)
    fit_L2_err_test_adap_list1_cubic.append(fit_L2_err_test1_list1_cubic)
    fit_L1_err_train_adap_list1_cubic.append(fit_L1_err_train1_list1_cubic)
    fit_L1_err_test_adap_list1_cubic.append(fit_L1_err_test1_list1_cubic)    
    
    fit_L2_err_train_ave_cubic = np.mean(torch.stack(fit_L2_err_train1_list1_cubic).numpy(),axis=(1,))
    fit_L2_err_test_ave_cubic = np.mean(torch.stack(fit_L2_err_test1_list1_cubic).numpy(),axis=(1,))
    
    fit_L2_err_train_adap_mean_list1_cubic.append(np.mean(fit_L2_err_train_ave_cubic))
    fit_L2_err_train_adap_std_list1_cubic.append(np.std(fit_L2_err_train_ave_cubic))
    fit_L2_err_test_adap_mean_list1_cubic.append(np.mean(fit_L2_err_test_ave_cubic))
    fit_L2_err_test_adap_std_list1_cubic.append(np.std(fit_L2_err_test_ave_cubic))
    
    print('\nL2 fitting error of training data (cubic): mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave_cubic),
                                                                         np.std(fit_L2_err_train_ave_cubic)))
    pred_window_test_adap_list1_cubic.append(pred_window_test1_list1_cubic)
    pred_window_test_adap_list1_cubic_pre.append(pred_window_test1_list1_cubic_pre)
    pred_L2_err_test_adap_list1_cubic.append(pred_L2_err_test1_list1_cubic)
    pred_L1_err_test_adap_list1_cubic.append(pred_L1_err_test1_list1_cubic)    
    
    pred_L2_err_test_ave_cubic = np.mean(torch.stack(pred_L2_err_test1_list1_cubic).numpy(),axis=(1,))
    
    pred_L2_err_test_adap_mean_list1_cubic.append(np.mean(pred_L2_err_test_ave_cubic))
    pred_L2_err_test_adap_std_list1_cubic.append(np.std(pred_L2_err_test_ave_cubic))
    
    print('\nL2 predicting error of testing data (cubic): mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave_cubic),
                                                                         np.std(pred_L2_err_test_ave_cubic)))
     
    

In [ ]:
# RNN_ODE_Adap, number of grids < 16

thres_list2 = [
    0.9, # 8, 2
    0, # 8, 2
    0.7, # 16, 2
              ]

rescale_const = 1

verbose = 2

num_grids_adap_list2 = []

odefunc_adap_list2, outputfunc_adap_list2 = [], []

# fitting errors
fit_window_train_adap_list2_cubic, fit_window_test_adap_list2_cubic = [], []
fit_L2_err_train_adap_list2_cubic, fit_L2_err_test_adap_list2_cubic = [], []
fit_L1_err_train_adap_list2_cubic, fit_L1_err_test_adap_list2_cubic = [], []

fit_L2_err_train_adap_mean_list2_cubic, fit_L2_err_test_adap_mean_list2_cubic = [], []
fit_L2_err_train_adap_std_list2_cubic, fit_L2_err_test_adap_std_list2_cubic = [], []


# predicting errors
pred_window_test_adap_list2_cubic = []
pred_window_test_adap_list2_cubic_pre = []
pred_L2_err_test_adap_list2_cubic = []
pred_L1_err_test_adap_list2_cubic = []

pred_L2_err_test_adap_mean_list2_cubic = []
pred_L2_err_test_adap_std_list2_cubic = []


for i, thres in enumerate(thres_list2):
    
#     if i <= 1:
#         num_fine_adap = 64
#         num_layers_adap = 4        
#     elif i <= 4:
#         num_fine_adap = 64
#         num_layers_adap = 3
#     else:
#         num_fine_adap = 64
#         num_layers_adap = 2
            
    if i <= 1:
        num_fine_adap = 8
        num_layers_adap = 2
    else:
        num_fine_adap = 16
        num_layers_adap = 2
        
        
        
    print('\n----------------------------------------------------------------\n')

    odefunc1_list2, outputfunc1_list2 = [], []
    
    fit_window_train1_list2_cubic, fit_window_test1_list2_cubic = [], []
    fit_L2_err_train1_list2_cubic, fit_L2_err_test1_list2_cubic = [], []
    fit_L1_err_train1_list2_cubic, fit_L1_err_test1_list2_cubic = [], []
    fit_window_train1_list2_linear, fit_window_test1_list2_linear = [], []
    fit_L2_err_train1_list2_linear, fit_L2_err_test1_list2_linear = [], []
    fit_L1_err_train1_list2_linear, fit_L1_err_test1_list2_linear = [], []

    pred_window_test1_list2_cubic = []
    pred_window_test1_list2_cubic_pre = []
    pred_L2_err_test1_list2_cubic = []
    pred_L1_err_test1_list2_cubic = []
    pred_window_test1_list2_linear = []
    pred_window_test1_list2_linear_pre = []
    pred_L2_err_test1_list2_linear = []
    pred_L1_err_test1_list2_linear = []
    
    num_grids_adap_list2_temp = []

    rep = 0
    while rep <= num_rep-1:
        
        print('\n------the %d-th replica:--------' % (rep+1))

        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]

        train_windows0_adap_buffer, train_ts0_adap_buffer, train_len0_adap_buffer, train_windows_adap_len = create_adap_data_buffer(
            train_windows0, train_ts0, thres, num_fine_adap, num_layers_adap)
        test_windows0_adap_buffer, test_ts0_adap_buffer, test_len0_adap_buffer, _ = create_adap_data_buffer(
            test_windows0, test_ts0, thres, num_fine_adap, num_layers_adap)
        # train_windows, 1st half
        train_windows0_adap_buffer_trunc, train_ts0_adap_buffer_trunc, _, _, train_time_index = create_adap_data_buffer(
            train_windows0[:,:(train_ts0.shape[1]+1)//2,:], train_ts0[:,:(train_ts0.shape[1]+1)//2], thres, num_fine_adap//2, num_layers_adap, return_index=1)
        # train_windows, 2nd half
        _, train_ts0_adap_buffer_trunc2, train_len0_adap_buffer_trunc2, _ = create_adap_data_buffer(
            train_windows0[:,-(train_ts0.shape[1]+1)//2:,:], train_ts0[:,-(train_ts0.shape[1]+1)//2:], thres, num_fine_adap//2, num_layers_adap, buffer_start_steps=0)
        # test_windows, 1st half
        test_windows0_adap_buffer_trunc, test_ts0_adap_buffer_trunc, _, _, test_time_index = create_adap_data_buffer(
            test_windows0[:,:(test_ts0.shape[1]+1)//2,:], test_ts0[:,:(test_ts0.shape[1]+1)//2], thres, num_fine_adap//2, num_layers_adap, return_index=1)
        # test_windows, 2nd half
        _, test_ts0_adap_buffer_trunc2, test_len0_adap_buffer_trunc2, _ = create_adap_data_buffer(
            test_windows0[:,-(test_ts0.shape[1]+1)//2:,:], test_ts0[:,-(test_ts0.shape[1]+1)//2:], thres, num_fine_adap//2, num_layers_adap, buffer_start_steps=0)

        num_grids_adap_list2_temp.append(train_windows_adap_len)
        
#         print(train_windows0[:,:(train_ts0.shape[1]+1)//2,:].shape)

        print('\nadaptive method, thres = %.2f, number of grids = %.1f (%.2e)\n' % (thres, train_windows_adap_len, train_windows_adap_len))
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        valid_windows0_adap_buffer = train_windows0_adap_buffer[valid_index,:,:]
        valid_ts0_adap_buffer = train_ts0_adap_buffer[valid_index,:]
        valid_windows0_adap_buffer_trunc = train_windows0_adap_buffer_trunc[valid_index,:,:]
        valid_ts0_adap_buffer_trunc = train_ts0_adap_buffer_trunc[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     
        train_loader_adap1_buffer = torch.utils.data.DataLoader(
            MyDataset_adap(train_windows=train_windows0_adap_buffer[train_index], train_ts=train_ts0_adap_buffer[train_index], 
                            train_data_len=np.array(train_len0_adap_buffer)[train_index]), 
            shuffle=True, batch_size=batch_size,) 
        input_steps_adap1_buffer = max(train_len0_adap_buffer)   
        
        print('train models:\n')
        
        if i <= 1:
            valid_ts_adap_trunc2, valid_len_adap_trunc2 = valid_ts0[:, (valid_ts0.shape[1]-1)//2::8], [(valid_ts0.shape[1]-1)//16+1]*test_ts0.shape[0]
        else:
            valid_ts_adap_trunc2, valid_len_adap_trunc2 = valid_ts0[:, (valid_ts0.shape[1]-1)//2::4], [(valid_ts0.shape[1]-1)//8+1]*test_ts0.shape[0]

        flag, odefunc_adap1_buffer, outputfunc_adap1_buffer = train_adap_models(
            train_loader_adap1_buffer, input_steps_adap1_buffer, verbose, 
            valid_window=valid_windows0, valid_ts=valid_ts0, valid_window_adap=valid_windows0_adap_buffer, 
            valid_ts_adap=valid_ts0_adap_buffer, valid_window_adap_trunc=valid_windows0_adap_buffer_trunc, 
            valid_ts_adap_trunc=valid_ts0_adap_buffer_trunc, buffer_start_steps=buffer_start_steps, 
            weight=[1,0], n_iter=n_iter, thres1=1.5e3, valid_len=np.array(train_len0_adap_buffer)[valid_index], 
            valid_ts_adap_trunc2=valid_ts_adap_trunc2, valid_len_adap_trunc2=valid_len_adap_trunc2,
            pred_len=(train_ts0.shape[1]-1)//2, rescale_const=rescale_const,
            n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )    
        
        if not flag:
            continue
        else:
            rep += 1
            
        odefunc1_list2.append(copy.deepcopy(odefunc_adap1_buffer))
        outputfunc1_list2.append(copy.deepcopy(outputfunc_adap1_buffer))
        
        if i <= 1:
            test_ts_adap_trunc2, test_len_adap_trunc2 = test_ts0[:, (test_ts0.shape[1]-1)//2::8], [(test_ts0.shape[1]-1)//16+1]*test_ts0.shape[0]
        else:
            test_ts_adap_trunc2, test_len_adap_trunc2 = test_ts0[:, (test_ts0.shape[1]-1)//2::4], [(test_ts0.shape[1]-1)//8+1]*test_ts0.shape[0]
        print(test_ts_adap_trunc2.shape)
        interp_kind = 'cubic'
        print('\nfitting error of training data (cubic):')
        fit_window_adap_train1, fit_L2_err_adap_train1, fit_L1_err_adap_train1 = fit_adap_models(
            odefunc_adap1_buffer, outputfunc_adap1_buffer, train_windows0, train_ts0, 
            train_windows0_adap_buffer, train_ts0_adap_buffer, train_len0_adap_buffer, buffer_start_steps,
            interp_kind=interp_kind, rescale_const=rescale_const, n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )   
        print('fitting error of testing data (cubic):')
        fit_window_adap_test1, fit_L2_err_adap_test1, fit_L1_err_adap_test1 = fit_adap_models(
            odefunc_adap1_buffer, outputfunc_adap1_buffer, test_windows0, test_ts0, 
            test_windows0_adap_buffer, test_ts0_adap_buffer, test_len0_adap_buffer, buffer_start_steps,
            interp_kind=interp_kind, rescale_const=rescale_const, n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )    
        fit_L2_err_train1_list2_cubic.append(fit_L2_err_adap_train1)
        fit_L2_err_test1_list2_cubic.append(fit_L2_err_adap_test1)
        fit_L1_err_train1_list2_cubic.append(fit_L1_err_adap_train1)
        fit_L1_err_test1_list2_cubic.append(fit_L1_err_adap_test1)
        fit_window_train1_list2_cubic.append(fit_window_adap_train1)
        fit_window_test1_list2_cubic.append(fit_window_adap_test1)

        print('predicting error of testing data (cubic):')
        pred_window_adap_test1_pre, pred_window_adap_test1, pred_L2_err_adap_test1, pred_L1_err_adap_test1 = pred_adap_models(
            odefunc_adap1_buffer, outputfunc_adap1_buffer, test_windows0, test_ts0, 
            test_windows0_adap_buffer_trunc, test_ts0_adap_buffer_trunc, rescale_const=rescale_const,
            ts_adap_buffer_trunc2=test_ts_adap_trunc2, len_adap_buffer_trunc2=test_len_adap_trunc2, 
            pred_len=(train_ts0.shape[1]-1)//2, buffer_start_steps=buffer_start_steps,interp_kind=interp_kind, 
            n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale
        )     

        pred_L2_err_test1_list2_cubic.append(pred_L2_err_adap_test1)
        pred_L1_err_test1_list2_cubic.append(pred_L1_err_adap_test1)
        pred_window_test1_list2_cubic.append(pred_window_adap_test1)
        pred_window_test1_list2_cubic_pre.append(pred_window_adap_test1_pre)
        

        
    odefunc_adap_list2.append(odefunc1_list2)
    outputfunc_adap_list2.append(outputfunc1_list2)
    num_grids_adap_list2.append(np.mean(num_grids_adap_list2_temp))
    
    fit_window_train_adap_list2_cubic.append(fit_window_train1_list2_cubic)
    fit_window_test_adap_list2_cubic.append(fit_window_test1_list2_cubic)
    fit_L2_err_train_adap_list2_cubic.append(fit_L2_err_train1_list2_cubic)
    fit_L2_err_test_adap_list2_cubic.append(fit_L2_err_test1_list2_cubic)
    fit_L1_err_train_adap_list2_cubic.append(fit_L1_err_train1_list2_cubic)
    fit_L1_err_test_adap_list2_cubic.append(fit_L1_err_test1_list2_cubic)    
    
    fit_L2_err_train_ave_cubic = np.mean(torch.stack(fit_L2_err_train1_list2_cubic).numpy(),axis=(1,))
    fit_L2_err_test_ave_cubic = np.mean(torch.stack(fit_L2_err_test1_list2_cubic).numpy(),axis=(1,))
    
    fit_L2_err_train_adap_mean_list2_cubic.append(np.mean(fit_L2_err_train_ave_cubic))
    fit_L2_err_train_adap_std_list2_cubic.append(np.std(fit_L2_err_train_ave_cubic))
    fit_L2_err_test_adap_mean_list2_cubic.append(np.mean(fit_L2_err_test_ave_cubic))
    fit_L2_err_test_adap_std_list2_cubic.append(np.std(fit_L2_err_test_ave_cubic))
    
    print('\nL2 fitting error of training data (cubic): mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave_cubic),
                                                                         np.std(fit_L2_err_train_ave_cubic)))
    pred_window_test_adap_list2_cubic.append(pred_window_test1_list2_cubic)
    pred_window_test_adap_list2_cubic_pre.append(pred_window_test1_list2_cubic_pre)
    pred_L2_err_test_adap_list2_cubic.append(pred_L2_err_test1_list2_cubic)
    pred_L1_err_test_adap_list2_cubic.append(pred_L1_err_test1_list2_cubic)    
    
    pred_L2_err_test_ave_cubic = np.mean(torch.stack(pred_L2_err_test1_list2_cubic).numpy(),axis=(1,))
    
    pred_L2_err_test_adap_mean_list2_cubic.append(np.mean(pred_L2_err_test_ave_cubic))
    pred_L2_err_test_adap_std_list2_cubic.append(np.std(pred_L2_err_test_ave_cubic))
    
    print('\nL2 predicting error of testing data (cubic): mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave_cubic),
                                                                         np.std(pred_L2_err_test_ave_cubic)))


In [ ]:
# compute errors

pred_L2_err_test_adap_list_mean_mat1 = np.zeros((num_rep, len(num_grids_adap_list1)))

for i in range(len(num_grids_adap_list1)):
    pred_L2_err_test_adap_list_mean_mat1[:,i] = torch.stack(pred_L2_err_test_adap_list1_cubic[i]).mean(axis=(1,)).numpy()

    
pred_L2_err_test_adap_list_mean_mat2 = np.zeros((num_rep, len(num_grids_adap_list2)))

for i in range(len(num_grids_adap_list2)):
    pred_L2_err_test_adap_list_mean_mat2[:,i] = torch.stack(pred_L2_err_test_adap_list2_cubic[i]).mean(axis=(1,)).numpy()
    


# RNN models

In [ ]:
# RNN, number of grids >= 16

num_grids_RNN_list1 = np.arange(16,68,4, dtype=int)+1#np.arange(24,102,6, dtype=int)+1

rnnfunc_RNN_list1 = []
fit_window_train_RNN_list1, fit_window_test_RNN_list1 = [], []
fit_L2_err_train_RNN_list1, fit_L2_err_test_RNN_list1 = [], []
fit_L1_err_train_RNN_list1, fit_L1_err_test_RNN_list1 = [], []

fit_L2_err_train_RNN_mean_list1, fit_L2_err_test_RNN_mean_list1 = [], []
fit_L2_err_train_RNN_std_list1, fit_L2_err_test_RNN_std_list1 = [], []

pred_window_train_RNN_list1, pred_window_test_RNN_list1 = [], []
pred_window_train_RNN_list1_pre, pred_window_test_RNN_list1_pre = [], []
pred_L2_err_train_RNN_list1, pred_L2_err_test_RNN_list1 = [], []
pred_L1_err_train_RNN_list1, pred_L1_err_test_RNN_list1 = [], []

pred_L2_err_train_RNN_mean_list1, pred_L2_err_test_RNN_mean_list1 = [], []
pred_L2_err_train_RNN_std_list1, pred_L2_err_test_RNN_std_list1 = [], []

rng = np.random.default_rng() 
    
batch_size = 64#128
verbose = 2
buffer_start_steps = 2
n_iter = 40

t1 = time.time()
index = np.arange(num_fine_grid+1)
for i, num_grids in enumerate(num_grids_RNN_list1):
    print('\n----------------------------------------------------------------\n')
    print('\nnon-adaptive method, number of grids = %d\n' % num_grids)
    index1 = np.round(np.arange(num_grids-1) * num_fine_grid/(num_grids-1))
    index1 = np.concatenate(([index1[0]]*buffer_start_steps, index1, [num_fine_grid])) 

    print('train models:\n')

    rnn1_list = []
    fit_window_train1_list, fit_window_test1_list = [], []
    fit_L2_err_train1_list, fit_L2_err_test1_list = [], []
    fit_L1_err_train1_list, fit_L1_err_test1_list = [], []

    pred_window_train1_list, pred_window_test1_list = [], []
    pred_window_train1_list_pre, pred_window_test1_list_pre = [], []
    pred_L2_err_train1_list, pred_L2_err_test1_list = [], []
    pred_L1_err_train1_list, pred_L1_err_test1_list = [], []

    rep = 0
    while rep <= num_rep-1:
        
        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]
                    
        train_ts1 = torch.tensor(np.linspace(train_ts0[:,0], train_ts0[:,-1], num_grids).T)
        train_ts1 = torch.concat((train_ts1[:,0].reshape(train_ts0.shape[0],1).repeat(1,buffer_start_steps), 
                                train_ts1),axis=1)
        train_windows1 = interpolate_window(train_windows0, train_ts0, train_ts1, 'cubic')
        
        if buffer_start_steps > 0:
            deltat = train_ts1[0][-1] - train_ts1[0][-2]
            train_ts1[:,:buffer_start_steps] = (train_ts1[:,:buffer_start_steps] + 
                                                torch.arange(-deltat*buffer_start_steps, 0, deltat))
            
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     

        train_loader1 = torch.utils.data.DataLoader(
            MyDataset(train_windows=train_windows1[train_index,:,:], train_ts=train_ts1[train_index,:]), 
            shuffle=True, batch_size=batch_size,) # rescale the data

        input_steps1 = train_windows1.shape[1]  

        print('\n------the %d-th replica:--------' % (rep+1))
        flag, rnn1 = train_RNN(train_loader1, input_steps1, valid_windows0, valid_ts0, num_grids, verbose, 
                                n_iter=n_iter, thres1=4e2, n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale )   
        if not flag:
            continue
        else:
            rep += 1
        
        print('\nfitting error of training data:')
        fit_window_train1, fit_L2_err_train1, fit_L1_err_train1 = fit_RNN_grids(
            rnn1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\nfitting error of testing data:')
        fit_window_test1, fit_L2_err_test1, fit_L1_err_test1 = fit_RNN_grids(
            rnn1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)


        rnn1_list.append((copy.deepcopy(rnn1)))
        
        fit_L2_err_train1_list.append(fit_L2_err_train1)
        fit_L2_err_test1_list.append(fit_L2_err_test1)
        fit_L1_err_train1_list.append(fit_L1_err_train1)
        fit_L1_err_test1_list.append(fit_L1_err_test1)
        fit_window_train1_list.append(fit_window_train1)
        fit_window_test1_list.append(fit_window_test1)

        print('\npredicting error of training data:')
        pred_window_train1_pre, pred_window_train1, pred_L2_err_train1, pred_L1_err_train1 = pred_RNN_grids(
            rnn1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\npredicting error of testing data:')
        pred_window_test1_pre, pred_window_test1, pred_L2_err_test1, pred_L1_err_test1 = pred_RNN_grids(
            rnn1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)

        pred_L2_err_train1_list.append(pred_L2_err_train1)
        pred_L2_err_test1_list.append(pred_L2_err_test1)
        pred_L1_err_train1_list.append(pred_L1_err_train1)
        pred_L1_err_test1_list.append(pred_L1_err_test1)
        pred_window_train1_list.append(pred_window_train1)
        pred_window_test1_list.append(pred_window_test1)
        pred_window_train1_list_pre.append(pred_window_train1_pre)
        pred_window_test1_list_pre.append(pred_window_test1_pre)

        
    rnnfunc_RNN_list1.append(rnn1_list)

    fit_window_train_RNN_list1.append(fit_window_train1_list)
    fit_window_test_RNN_list1.append(fit_window_test1_list)
    fit_L2_err_train_RNN_list1.append(fit_L2_err_train1_list)
    fit_L2_err_test_RNN_list1.append(fit_L2_err_test1_list)
    fit_L1_err_train_RNN_list1.append(fit_L1_err_train1_list)
    fit_L1_err_test_RNN_list1.append(fit_L1_err_test1_list)    
    
    fit_L2_err_train_ave = np.mean(torch.stack(fit_L2_err_train1_list).numpy(),axis=(1,))
    fit_L2_err_test_ave = np.mean(torch.stack(fit_L2_err_test1_list).numpy(),axis=(1,))
    fit_L2_err_train_RNN_mean_list1.append(np.mean(fit_L2_err_train_ave))
    fit_L2_err_train_RNN_std_list1.append(np.std(fit_L2_err_train_ave))
    fit_L2_err_test_RNN_mean_list1.append(np.mean(fit_L2_err_test_ave))
    fit_L2_err_test_RNN_std_list1.append(np.std(fit_L2_err_test_ave))
    
    print('\nL2 fitting error of training data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave),
                                                                         np.std(fit_L2_err_train_ave)))
    print('\nL2 fitting error of testing data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_test_ave),
                                                                         np.std(fit_L2_err_test_ave)))
    pred_window_train_RNN_list1.append(pred_window_train1_list)
    pred_window_test_RNN_list1.append(pred_window_test1_list)
    pred_window_train_RNN_list1_pre.append(pred_window_train1_list_pre)
    pred_window_test_RNN_list1_pre.append(pred_window_test1_list_pre)
    pred_L2_err_train_RNN_list1.append(pred_L2_err_train1_list)
    pred_L2_err_test_RNN_list1.append(pred_L2_err_test1_list)
    pred_L1_err_train_RNN_list1.append(pred_L1_err_train1_list)
    pred_L1_err_test_RNN_list1.append(pred_L1_err_test1_list)    
    
    pred_L2_err_train_ave = np.mean(torch.stack(pred_L2_err_train1_list).numpy(),axis=(1,))
    pred_L2_err_test_ave = np.mean(torch.stack(pred_L2_err_test1_list).numpy(),axis=(1,))
    pred_L2_err_train_RNN_mean_list1.append(np.mean(pred_L2_err_train_ave))
    pred_L2_err_train_RNN_std_list1.append(np.std(pred_L2_err_train_ave))
    pred_L2_err_test_RNN_mean_list1.append(np.mean(pred_L2_err_test_ave))
    pred_L2_err_test_RNN_std_list1.append(np.std(pred_L2_err_test_ave))

    print('\nL2 predicting error of training data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_train_ave),
                                                                         np.std(pred_L2_err_train_ave)))
    print('\nL2 predicting error of testing data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave),
                                                                         np.std(pred_L2_err_test_ave)))
    
t2 = time.time()
print('training model takes %.2f s' % (t2-t1) )
    

In [ ]:
# RNN, number of grids < 16


num_grids_RNN_list2 = np.array([6, 8, 12])+1#np.arange(24,102,6, dtype=int)+1

rnnfunc_RNN_list2 = []
fit_window_train_RNN_list2, fit_window_test_RNN_list2 = [], []
fit_L2_err_train_RNN_list2, fit_L2_err_test_RNN_list2 = [], []
fit_L1_err_train_RNN_list2, fit_L1_err_test_RNN_list2 = [], []

fit_L2_err_train_RNN_mean_list2, fit_L2_err_test_RNN_mean_list2 = [], []
fit_L2_err_train_RNN_std_list2, fit_L2_err_test_RNN_std_list2 = [], []

pred_window_train_RNN_list2, pred_window_test_RNN_list2 = [], []
pred_window_train_RNN_list2_pre, pred_window_test_RNN_list2_pre = [], []
pred_L2_err_train_RNN_list2, pred_L2_err_test_RNN_list2 = [], []
pred_L1_err_train_RNN_list2, pred_L1_err_test_RNN_list2 = [], []

pred_L2_err_train_RNN_mean_list2, pred_L2_err_test_RNN_mean_list2 = [], []
pred_L2_err_train_RNN_std_list2, pred_L2_err_test_RNN_std_list2 = [], []

rng = np.random.default_rng() 
    
batch_size = 64#128
verbose = 2
buffer_start_steps = 2

t1 = time.time()
index = np.arange(num_fine_grid+1)
for i, num_grids in enumerate(num_grids_RNN_list2):
    print('\n----------------------------------------------------------------\n')
    print('\nnon-adaptive method, number of grids = %d\n' % num_grids)
    index1 = np.round(np.arange(num_grids-1) * num_fine_grid/(num_grids-1))
    index1 = np.concatenate(([index1[0]]*buffer_start_steps, index1, [num_fine_grid])) 

    print('train models:\n')

    rnn1_list = []
    fit_window_train1_list, fit_window_test1_list = [], []
    fit_L2_err_train1_list, fit_L2_err_test1_list = [], []
    fit_L1_err_train1_list, fit_L1_err_test1_list = [], []

    pred_window_train1_list, pred_window_test1_list = [], []
    pred_window_train1_list_pre, pred_window_test1_list_pre = [], []
    pred_L2_err_train1_list, pred_L2_err_test1_list = [], []
    pred_L1_err_train1_list, pred_L1_err_test1_list = [], []

    rep = 0
    while rep <= num_rep-1:
        
        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]
                    
        train_ts1 = torch.tensor(np.linspace(train_ts0[:,0], train_ts0[:,-1], num_grids).T)
        train_ts1 = torch.concat((train_ts1[:,0].reshape(train_ts0.shape[0],1).repeat(1,buffer_start_steps), 
                                train_ts1),axis=1)
        train_windows1 = interpolate_window(train_windows0, train_ts0, train_ts1, 'cubic')
        
        if buffer_start_steps > 0:
            deltat = train_ts1[0][-1] - train_ts1[0][-2]
            train_ts1[:,:buffer_start_steps] = (train_ts1[:,:buffer_start_steps] + 
                                                torch.arange(-deltat*buffer_start_steps, 0, deltat))
            
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     

        train_loader1 = torch.utils.data.DataLoader(
            MyDataset(train_windows=train_windows1[train_index,:,:], train_ts=train_ts1[train_index,:]), 
            shuffle=True, batch_size=batch_size) # rescale the data

        input_steps1 = train_windows1.shape[1]  

        print('\n------the %d-th replica:--------' % (rep+1))
        flag, rnn1 = train_RNN(train_loader1, input_steps1, valid_windows0, valid_ts0, num_grids, verbose, 
                                n_iter=n_iter, thres1=4e2, n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale)    
        if not flag:
            continue
        else:
            rep += 1
        
        print('\nfitting error of training data:')
        fit_window_train1, fit_L2_err_train1, fit_L1_err_train1 = fit_RNN_grids(
            rnn1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\nfitting error of testing data:')
        fit_window_test1, fit_L2_err_test1, fit_L1_err_test1 = fit_RNN_grids(
            rnn1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)


        rnn1_list.append((copy.deepcopy(rnn1)))
        
        fit_L2_err_train1_list.append(fit_L2_err_train1)
        fit_L2_err_test1_list.append(fit_L2_err_test1)
        fit_L1_err_train1_list.append(fit_L1_err_train1)
        fit_L1_err_test1_list.append(fit_L1_err_test1)
        fit_window_train1_list.append(fit_window_train1)
        fit_window_test1_list.append(fit_window_test1)

        print('\npredicting error of training data:')
        pred_window_train1_pre, pred_window_train1, pred_L2_err_train1, pred_L1_err_train1 = pred_RNN_grids(
            rnn1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\npredicting error of testing data:')
        pred_window_test1_pre, pred_window_test1, pred_L2_err_test1, pred_L1_err_test1 = pred_RNN_grids(
            rnn1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)

        pred_L2_err_train1_list.append(pred_L2_err_train1)
        pred_L2_err_test1_list.append(pred_L2_err_test1)
        pred_L1_err_train1_list.append(pred_L1_err_train1)
        pred_L1_err_test1_list.append(pred_L1_err_test1)
        pred_window_train1_list.append(pred_window_train1)
        pred_window_test1_list.append(pred_window_test1)
        pred_window_train1_list_pre.append(pred_window_train1_pre)
        pred_window_test1_list_pre.append(pred_window_test1_pre)

        
    rnnfunc_RNN_list2.append(rnn1_list)

    fit_window_train_RNN_list2.append(fit_window_train1_list)
    fit_window_test_RNN_list2.append(fit_window_test1_list)
    fit_L2_err_train_RNN_list2.append(fit_L2_err_train1_list)
    fit_L2_err_test_RNN_list2.append(fit_L2_err_test1_list)
    fit_L1_err_train_RNN_list2.append(fit_L1_err_train1_list)
    fit_L1_err_test_RNN_list2.append(fit_L1_err_test1_list)    
    
    fit_L2_err_train_ave = np.mean(torch.stack(fit_L2_err_train1_list).numpy(),axis=(1,))
    fit_L2_err_test_ave = np.mean(torch.stack(fit_L2_err_test1_list).numpy(),axis=(1,))
    fit_L2_err_train_RNN_mean_list2.append(np.mean(fit_L2_err_train_ave))
    fit_L2_err_train_RNN_std_list2.append(np.std(fit_L2_err_train_ave))
    fit_L2_err_test_RNN_mean_list2.append(np.mean(fit_L2_err_test_ave))
    fit_L2_err_test_RNN_std_list2.append(np.std(fit_L2_err_test_ave))
    
    print('\nL2 fitting error of training data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave),
                                                                         np.std(fit_L2_err_train_ave)))
    print('\nL2 fitting error of testing data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_test_ave),
                                                                         np.std(fit_L2_err_test_ave)))
    pred_window_train_RNN_list2.append(pred_window_train1_list)
    pred_window_test_RNN_list2.append(pred_window_test1_list)
    pred_window_train_RNN_list2_pre.append(pred_window_train1_list_pre)
    pred_window_test_RNN_list2_pre.append(pred_window_test1_list_pre)
    pred_L2_err_train_RNN_list2.append(pred_L2_err_train1_list)
    pred_L2_err_test_RNN_list2.append(pred_L2_err_test1_list)
    pred_L1_err_train_RNN_list2.append(pred_L1_err_train1_list)
    pred_L1_err_test_RNN_list2.append(pred_L1_err_test1_list)    
    
    pred_L2_err_train_ave = np.mean(torch.stack(pred_L2_err_train1_list).numpy(),axis=(1,))
    pred_L2_err_test_ave = np.mean(torch.stack(pred_L2_err_test1_list).numpy(),axis=(1,))
    pred_L2_err_train_RNN_mean_list2.append(np.mean(pred_L2_err_train_ave))
    pred_L2_err_train_RNN_std_list2.append(np.std(pred_L2_err_train_ave))
    pred_L2_err_test_RNN_mean_list2.append(np.mean(pred_L2_err_test_ave))
    pred_L2_err_test_RNN_std_list2.append(np.std(pred_L2_err_test_ave))

    print('\nL2 predicting error of training data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_train_ave),
                                                                         np.std(pred_L2_err_train_ave)))
    print('\nL2 predicting error of testing data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave),
                                                                         np.std(pred_L2_err_test_ave)))
    
t2 = time.time()
print('training model takes %.2f s' % (t2-t1) )
    

In [ ]:
# compute errors

pred_L2_err_test_RNN_list_mean_mat1 = np.zeros((num_rep, len(num_grids_RNN_list1)))

for i in range(len(num_grids_RNN_list1)):
    pred_L2_err_test_RNN_list_mean_mat1[:,i] = torch.stack(pred_L2_err_test_RNN_list1[i]).mean(axis=(1,)).numpy()
    
pred_L2_err_test_RNN_list_mean_mat2 = np.zeros((num_rep, len(num_grids_RNN_list2)))

for i in range(len(num_grids_RNN_list2)):
    pred_L2_err_test_RNN_list_mean_mat2[:,i] = torch.stack(pred_L2_err_test_RNN_list2[i]).mean(axis=(1,)).numpy()
    


# LSTM models

In [ ]:
# LSTM, number of grids >= 16

num_grids_LSTM_list1 = np.arange(16,68,4, dtype=int)+1#np.arange(24,102,6, dtype=int)+1

lstmfunc_LSTM_list1 = []
fit_window_train_LSTM_list1, fit_window_test_LSTM_list1 = [], []
fit_L2_err_train_LSTM_list1, fit_L2_err_test_LSTM_list1 = [], []
fit_L1_err_train_LSTM_list1, fit_L1_err_test_LSTM_list1 = [], []

fit_L2_err_train_LSTM_mean_list1, fit_L2_err_test_LSTM_mean_list1 = [], []
fit_L2_err_train_LSTM_std_list1, fit_L2_err_test_LSTM_std_list1 = [], []

pred_window_train_LSTM_list1, pred_window_test_LSTM_list1 = [], []
pred_window_train_LSTM_list1_pre, pred_window_test_LSTM_list1_pre = [], []
pred_L2_err_train_LSTM_list1, pred_L2_err_test_LSTM_list1 = [], []
pred_L1_err_train_LSTM_list1, pred_L1_err_test_LSTM_list1 = [], []

pred_L2_err_train_LSTM_mean_list1, pred_L2_err_test_LSTM_mean_list1 = [], []
pred_L2_err_train_LSTM_std_list1, pred_L2_err_test_LSTM_std_list1 = [], []

rng = np.random.default_rng() 
    

batch_size = 64#128
verbose = 2


t1 = time.time()
index = np.arange(num_fine_grid+1)
for i, num_grids in enumerate(num_grids_LSTM_list1):
    print('\n----------------------------------------------------------------\n')
    print('\nnon-adaptive method, number of grids = %d\n' % num_grids)
    index1 = np.round(np.arange(num_grids-1) * num_fine_grid/(num_grids-1))
    index1 = np.concatenate(([index1[0]]*buffer_start_steps, index1, [num_fine_grid])) 

    print('train models:\n')

    LSTM1_list = []
    fit_window_train1_list, fit_window_test1_list = [], []
    fit_L2_err_train1_list, fit_L2_err_test1_list = [], []
    fit_L1_err_train1_list, fit_L1_err_test1_list = [], []

    pred_window_train1_list, pred_window_test1_list = [], []
    pred_window_train1_list_pre, pred_window_test1_list_pre = [], []
    pred_L2_err_train1_list, pred_L2_err_test1_list = [], []
    pred_L1_err_train1_list, pred_L1_err_test1_list = [], []

    rep = 0
    while rep <= num_rep-1:
        
        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]
                    
        train_ts1 = torch.tensor(np.linspace(train_ts0[:,0], train_ts0[:,-1], num_grids).T)
        train_ts1 = torch.concat((train_ts1[:,0].reshape(train_ts0.shape[0],1).repeat(1,buffer_start_steps), 
                                train_ts1),axis=1)
        train_windows1 = interpolate_window(train_windows0, train_ts0, train_ts1, 'cubic')
        
        if buffer_start_steps > 0:
            deltat = train_ts1[0][-1] - train_ts1[0][-2]
            train_ts1[:,:buffer_start_steps] = (train_ts1[:,:buffer_start_steps] + 
                                                torch.arange(-deltat*buffer_start_steps, 0, deltat))
            
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     

        train_loader1 = torch.utils.data.DataLoader(
            MyDataset(train_windows=train_windows1[train_index,:,:], train_ts=train_ts1[train_index,:]), 
            shuffle=True, batch_size=batch_size,) # rescale the data

        input_steps1 = train_windows1.shape[1]  

        print('\n------the %d-th replica:--------' % (rep+1))
        flag, lstm1 = train_LSTM(train_loader1, input_steps1, valid_windows0, valid_ts0, num_grids, 
                                  verbose, n_iter=n_iter, thres1=6e2, thres2=5e2,
                                  n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale)    
        if not flag:
            continue
        else:
            rep += 1
        
        print('\nfitting error of training data:')
        fit_window_train1, fit_L2_err_train1, fit_L1_err_train1 = fit_LSTM_grids(
            lstm1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\nfitting error of testing data:')
        fit_window_test1, fit_L2_err_test1, fit_L1_err_test1 = fit_LSTM_grids(
            lstm1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)


        LSTM1_list.append(copy.deepcopy(lstm1))
        
        fit_L2_err_train1_list.append(fit_L2_err_train1)
        fit_L2_err_test1_list.append(fit_L2_err_test1)
        fit_L1_err_train1_list.append(fit_L1_err_train1)
        fit_L1_err_test1_list.append(fit_L1_err_test1)
        fit_window_train1_list.append(fit_window_train1)
        fit_window_test1_list.append(fit_window_test1)

        print('\npredicting error of training data:')
        pred_window_train1_pre, pred_window_train1, pred_L2_err_train1, pred_L1_err_train1 = pred_LSTM_grids(
            lstm1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\npredicting error of testing data:')
        pred_window_test1_pre, pred_window_test1, pred_L2_err_test1, pred_L1_err_test1 = pred_LSTM_grids(
            lstm1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)

        pred_L2_err_train1_list.append(pred_L2_err_train1)
        pred_L2_err_test1_list.append(pred_L2_err_test1)
        pred_L1_err_train1_list.append(pred_L1_err_train1)
        pred_L1_err_test1_list.append(pred_L1_err_test1)
        pred_window_train1_list.append(pred_window_train1)
        pred_window_test1_list.append(pred_window_test1)
        pred_window_train1_list_pre.append(pred_window_train1_pre)
        pred_window_test1_list_pre.append(pred_window_test1_pre)

    lstmfunc_LSTM_list1.append(LSTM1_list)

    fit_window_train_LSTM_list1.append(fit_window_train1_list)
    fit_window_test_LSTM_list1.append(fit_window_test1_list)
    fit_L2_err_train_LSTM_list1.append(fit_L2_err_train1_list)
    fit_L2_err_test_LSTM_list1.append(fit_L2_err_test1_list)
    fit_L1_err_train_LSTM_list1.append(fit_L1_err_train1_list)
    fit_L1_err_test_LSTM_list1.append(fit_L1_err_test1_list)    
    
    fit_L2_err_train_ave = np.mean(torch.stack(fit_L2_err_train1_list).numpy(),axis=(1,))
    fit_L2_err_test_ave = np.mean(torch.stack(fit_L2_err_test1_list).numpy(),axis=(1,))
    fit_L2_err_train_LSTM_mean_list1.append(np.mean(fit_L2_err_train_ave))
    fit_L2_err_train_LSTM_std_list1.append(np.std(fit_L2_err_train_ave))
    fit_L2_err_test_LSTM_mean_list1.append(np.mean(fit_L2_err_test_ave))
    fit_L2_err_test_LSTM_std_list1.append(np.std(fit_L2_err_test_ave))
    
    print('\nL2 fitting error of training data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave),
                                                                         np.std(fit_L2_err_train_ave)))
    print('\nL2 fitting error of testing data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_test_ave),
                                                                         np.std(fit_L2_err_test_ave)))

    pred_window_train_LSTM_list1.append(pred_window_train1_list)
    pred_window_test_LSTM_list1.append(pred_window_test1_list)
    pred_window_train_LSTM_list1_pre.append(pred_window_train1_list_pre)
    pred_window_test_LSTM_list1_pre.append(pred_window_test1_list_pre)
    pred_L2_err_train_LSTM_list1.append(pred_L2_err_train1_list)
    pred_L2_err_test_LSTM_list1.append(pred_L2_err_test1_list)
    pred_L1_err_train_LSTM_list1.append(pred_L1_err_train1_list)
    pred_L1_err_test_LSTM_list1.append(pred_L1_err_test1_list)    
    
    pred_L2_err_train_ave = np.mean(torch.stack(pred_L2_err_train1_list).numpy(),axis=(1,))
    pred_L2_err_test_ave = np.mean(torch.stack(pred_L2_err_test1_list).numpy(),axis=(1,))
    pred_L2_err_train_LSTM_mean_list1.append(np.mean(pred_L2_err_train_ave))
    pred_L2_err_train_LSTM_std_list1.append(np.std(pred_L2_err_train_ave))
    pred_L2_err_test_LSTM_mean_list1.append(np.mean(pred_L2_err_test_ave))
    pred_L2_err_test_LSTM_std_list1.append(np.std(pred_L2_err_test_ave))

    print('\nL2 predicting error of training data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_train_ave),
                                                                         np.std(pred_L2_err_train_ave)))
    print('\nL2 predicting error of testing data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave),
                                                                         np.std(pred_L2_err_test_ave)))

t2 = time.time()
print('training model takes %.2f s' % (t2-t1) )
    

In [ ]:
# LSTM, number of grids < 16

num_grids_LSTM_list2 = np.array([6,8,12])+1#np.arange(24,102,6, dtype=int)+1

lstmfunc_LSTM_list2 = []
fit_window_train_LSTM_list2, fit_window_test_LSTM_list2 = [], []
fit_L2_err_train_LSTM_list2, fit_L2_err_test_LSTM_list2 = [], []
fit_L1_err_train_LSTM_list2, fit_L1_err_test_LSTM_list2 = [], []

fit_L2_err_train_LSTM_mean_list2, fit_L2_err_test_LSTM_mean_list2 = [], []
fit_L2_err_train_LSTM_std_list2, fit_L2_err_test_LSTM_std_list2 = [], []

pred_window_train_LSTM_list2, pred_window_test_LSTM_list2 = [], []
pred_window_train_LSTM_list2_pre, pred_window_test_LSTM_list2_pre = [], []
pred_L2_err_train_LSTM_list2, pred_L2_err_test_LSTM_list2 = [], []
pred_L1_err_train_LSTM_list2, pred_L1_err_test_LSTM_list2 = [], []

pred_L2_err_train_LSTM_mean_list2, pred_L2_err_test_LSTM_mean_list2 = [], []
pred_L2_err_train_LSTM_std_list2, pred_L2_err_test_LSTM_std_list2 = [], []

rng = np.random.default_rng() 
    

batch_size = 64#128
verbose = 2


t1 = time.time()
index = np.arange(num_fine_grid+1)
for i, num_grids in enumerate(num_grids_LSTM_list2):
    print('\n----------------------------------------------------------------\n')
    print('\nnon-adaptive method, number of grids = %d\n' % num_grids)
    index1 = np.round(np.arange(num_grids-1) * num_fine_grid/(num_grids-1))
    index1 = np.concatenate(([index1[0]]*buffer_start_steps, index1, [num_fine_grid])) 

    print('train models:\n')

    LSTM1_list = []
    fit_window_train1_list, fit_window_test1_list = [], []
    fit_L2_err_train1_list, fit_L2_err_test1_list = [], []
    fit_L1_err_train1_list, fit_L1_err_test1_list = [], []

    pred_window_train1_list, pred_window_test1_list = [], []
    pred_window_train1_list_pre, pred_window_test1_list_pre = [], []
    pred_L2_err_train1_list, pred_L2_err_test1_list = [], []
    pred_L1_err_train1_list, pred_L1_err_test1_list = [], []

    rep = 0
    while rep <= num_rep-1:
        
        train_windows0 = train_windows0_list[rep]
        train_ts0 = train_ts0_list[rep]
        test_windows0 = test_windows0_list[rep]
        test_ts0 = test_ts0_list[rep]
                    
        train_ts1 = torch.tensor(np.linspace(train_ts0[:,0], train_ts0[:,-1], num_grids).T)
        train_ts1 = torch.concat((train_ts1[:,0].reshape(train_ts0.shape[0],1).repeat(1,buffer_start_steps), 
                                train_ts1),axis=1)
        train_windows1 = interpolate_window(train_windows0, train_ts0, train_ts1, 'cubic')
        
        if buffer_start_steps > 0:
            deltat = train_ts1[0][-1] - train_ts1[0][-2]
            train_ts1[:,:buffer_start_steps] = (train_ts1[:,:buffer_start_steps] + 
                                                torch.arange(-deltat*buffer_start_steps, 0, deltat))
            
        valid_index = np.sort(npr.choice(train_windows0.shape[0], 50, replace=False))
        valid_windows0 = train_windows0[valid_index,:,:]
        valid_ts0 = train_ts0[valid_index,:]
        
        train_index = np.delete(np.arange(train_windows0.shape[0]), valid_index)     

        train_loader1 = torch.utils.data.DataLoader(
            MyDataset(train_windows=train_windows1[train_index,:,:], train_ts=train_ts1[train_index,:]), 
            shuffle=True, batch_size=batch_size,) # rescale the data

        input_steps1 = train_windows1.shape[1]  

        print('\n------the %d-th replica:--------' % (rep+1))
        flag, lstm1 = train_LSTM(train_loader1, input_steps1, valid_windows0, valid_ts0, num_grids, 
                                verbose, n_iter=n_iter, thres1=6e2, thres2=5e2,
                                n_latent=n_latent, obs_dim=obs_dim, time_scale=time_scale)    
        if not flag:
            continue
        else:
            rep += 1
        
        print('\nfitting error of training data:')
        fit_window_train1, fit_L2_err_train1, fit_L1_err_train1 = fit_LSTM_grids(
            lstm1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\nfitting error of testing data:')
        fit_window_test1, fit_L2_err_test1, fit_L1_err_test1 = fit_LSTM_grids(
            lstm1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)


        LSTM1_list.append(copy.deepcopy(lstm1))
        
        fit_L2_err_train1_list.append(fit_L2_err_train1)
        fit_L2_err_test1_list.append(fit_L2_err_test1)
        fit_L1_err_train1_list.append(fit_L1_err_train1)
        fit_L1_err_test1_list.append(fit_L1_err_test1)
        fit_window_train1_list.append(fit_window_train1)
        fit_window_test1_list.append(fit_window_test1)

        print('\npredicting error of training data:')
        pred_window_train1_pre, pred_window_train1, pred_L2_err_train1, pred_L1_err_train1 = pred_LSTM_grids(
            lstm1, train_windows0, train_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)
        print('\npredicting error of testing data:')
        pred_window_test1_pre, pred_window_test1, pred_L2_err_test1, pred_L1_err_test1 = pred_LSTM_grids(
            lstm1, test_windows0, test_ts0, num_grids, buffer_start_steps=buffer_start_steps,
            n_latent=n_latent, obs_dim=obs_dim)

        pred_L2_err_train1_list.append(pred_L2_err_train1)
        pred_L2_err_test1_list.append(pred_L2_err_test1)
        pred_L1_err_train1_list.append(pred_L1_err_train1)
        pred_L1_err_test1_list.append(pred_L1_err_test1)
        pred_window_train1_list.append(pred_window_train1)
        pred_window_test1_list.append(pred_window_test1)
        pred_window_train1_list_pre.append(pred_window_train1_pre)
        pred_window_test1_list_pre.append(pred_window_test1_pre)

    lstmfunc_LSTM_list2.append(LSTM1_list)

    fit_window_train_LSTM_list2.append(fit_window_train1_list)
    fit_window_test_LSTM_list2.append(fit_window_test1_list)
    fit_L2_err_train_LSTM_list2.append(fit_L2_err_train1_list)
    fit_L2_err_test_LSTM_list2.append(fit_L2_err_test1_list)
    fit_L1_err_train_LSTM_list2.append(fit_L1_err_train1_list)
    fit_L1_err_test_LSTM_list2.append(fit_L1_err_test1_list)    
    
    fit_L2_err_train_ave = np.mean(torch.stack(fit_L2_err_train1_list).numpy(),axis=(1,))
    fit_L2_err_test_ave = np.mean(torch.stack(fit_L2_err_test1_list).numpy(),axis=(1,))
    fit_L2_err_train_LSTM_mean_list2.append(np.mean(fit_L2_err_train_ave))
    fit_L2_err_train_LSTM_std_list2.append(np.std(fit_L2_err_train_ave))
    fit_L2_err_test_LSTM_mean_list2.append(np.mean(fit_L2_err_test_ave))
    fit_L2_err_test_LSTM_std_list2.append(np.std(fit_L2_err_test_ave))
    
    print('\nL2 fitting error of training data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_train_ave),
                                                                         np.std(fit_L2_err_train_ave)))
    print('\nL2 fitting error of testing data: mean = %.2e, std = %.2e' % (np.mean(fit_L2_err_test_ave),
                                                                         np.std(fit_L2_err_test_ave)))

    pred_window_train_LSTM_list2.append(pred_window_train1_list)
    pred_window_test_LSTM_list2.append(pred_window_test1_list)
    pred_window_train_LSTM_list2_pre.append(pred_window_train1_list_pre)
    pred_window_test_LSTM_list2_pre.append(pred_window_test1_list_pre)
    pred_L2_err_train_LSTM_list2.append(pred_L2_err_train1_list)
    pred_L2_err_test_LSTM_list2.append(pred_L2_err_test1_list)
    pred_L1_err_train_LSTM_list2.append(pred_L1_err_train1_list)
    pred_L1_err_test_LSTM_list2.append(pred_L1_err_test1_list)    
    
    pred_L2_err_train_ave = np.mean(torch.stack(pred_L2_err_train1_list).numpy(),axis=(1,))
    pred_L2_err_test_ave = np.mean(torch.stack(pred_L2_err_test1_list).numpy(),axis=(1,))
    pred_L2_err_train_LSTM_mean_list2.append(np.mean(pred_L2_err_train_ave))
    pred_L2_err_train_LSTM_std_list2.append(np.std(pred_L2_err_train_ave))
    pred_L2_err_test_LSTM_mean_list2.append(np.mean(pred_L2_err_test_ave))
    pred_L2_err_test_LSTM_std_list2.append(np.std(pred_L2_err_test_ave))

    print('\nL2 predicting error of training data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_train_ave),
                                                                         np.std(pred_L2_err_train_ave)))
    print('\nL2 predicting error of testing data: mean = %.2e, std = %.2e' % (np.mean(pred_L2_err_test_ave),
                                                                         np.std(pred_L2_err_test_ave)))

t2 = time.time()
print('training model takes %.2f s' % (t2-t1) )
    

In [ ]:
# compute errors

pred_L2_err_test_LSTM_list_mean_mat1 = np.zeros((num_rep, len(num_grids_LSTM_list1)))

for i in range(len(num_grids_LSTM_list1)):
    pred_L2_err_test_LSTM_list_mean_mat1[:,i] = torch.stack(pred_L2_err_test_LSTM_list1[i]).mean(axis=(1,)).numpy()


pred_L2_err_test_LSTM_list_mean_mat2 = np.zeros((num_rep, len(num_grids_LSTM_list2)))

for i in range(len(num_grids_LSTM_list2)):
    pred_L2_err_test_LSTM_list_mean_mat2[:,i] = torch.stack(pred_L2_err_test_LSTM_list2[i]).mean(axis=(1,)).numpy()


# Plots

In [ ]:
# box plot

alpha = 0.5
index1 = [0,1,2,3,4,5,6,7,8,9,10,11,12]
# plt.figure(figsize=(7,5))
fig, ax = plt.subplots(figsize=(8,6))
linewidth = 2

cols = ['tab:green'] * 13
xticks = np.concatenate((np.array(num_grids_non_adap_list2), np.array(num_grids_non_adap_list1)))
err_temp = np.concatenate((np.array(pred_L2_err_test_non_adap_list_mean_mat2), np.array(pred_L2_err_test_non_adap_list_mean_mat1)), axis=1)
bp = ax.boxplot(err_temp, positions=xticks,
           widths = 0.4, whis=0.5, whiskerprops={'color' : 'tab:green'}, 
                medianprops={'color' : 'tab:green', 'linewidth': 3},
                boxprops={'color' : 'tab:green'},
                flierprops={'markersize':0.01})
res = {key : [v.get_ydata() for v in value] for key, value in bp.items()}
ax.plot(xticks, np.array(res['medians'])[:,0], 'x-', color='tab:green',
        label='RNNODE', alpha=alpha, linewidth=linewidth)
for b, c in zip(bp['boxes'], cols):
    b.set_alpha(alpha)
print(np.array(res['medians'])[:,0])


cols = ['tab:red'] * 13
xticks = np.concatenate((np.array(num_grids_non_adap_list2), np.array(num_grids_non_adap_list1)))
err_temp = np.concatenate((np.array(pred_L2_err_test_adap_list_mean_mat2), np.array(pred_L2_err_test_adap_list_mean_mat1)), axis=1)
bp = ax.boxplot(err_temp, positions=xticks,
           widths = 0.4, whis=0.5, whiskerprops={'color' : 'tab:red'}, 
                medianprops={'color' : 'tab:red', 'linewidth': 3},
                boxprops={'color' : 'tab:red'},
                flierprops={'markersize':0.01})
res = {key : [v.get_ydata() for v in value] for key, value in bp.items()}
ax.plot(xticks, np.array(res['medians'])[:,0], 'x-', color='tab:red',
        label='RNN-ODE-Adap', alpha=alpha, linewidth=linewidth)
for b, c in zip(bp['boxes'], cols):
    b.set_alpha(alpha)


cols = ['tab:blue'] * 13
xticks = np.concatenate((np.array(num_grids_RNN_list2), np.array(num_grids_RNN_list1)))
err_temp = np.concatenate((np.array(pred_L2_err_test_RNN_list_mean_mat2), np.array(pred_L2_err_test_RNN_list_mean_mat1)), axis=1)
bp = ax.boxplot(err_temp, positions=xticks,
                 widths = 0.4, whis=0.01, whiskerprops={'color' : 'tab:blue'}, 
                 medianprops={'color' : 'tab:blue', 'linewidth': 3},
                 boxprops={'color' : 'tab:blue'},
                 flierprops={'markersize':0.01}
                )
res = {key : [v.get_ydata() for v in value] for key, value in bp.items()}
ax.plot(xticks, np.array(res['medians'])[:,0], 'x-', color='tab:blue',
        label='RNN', alpha=alpha, linewidth=linewidth)
for b, c in zip(bp['boxes'], cols):
    b.set_alpha(alpha)
print(np.array(res['medians'])[:,0])

cols = ['tab:orange'] * 13
xticks = np.concatenate((np.array(num_grids_LSTM_list2), np.array(num_grids_LSTM_list1)))
err_temp = np.concatenate((np.array(pred_L2_err_test_LSTM_list_mean_mat2), np.array(pred_L2_err_test_LSTM_list_mean_mat1)), axis=1)
bp = ax.boxplot(err_temp, positions=xticks,
                 widths = 0.4, whis=0.5, whiskerprops={'color' : 'tab:orange'}, 
                 medianprops={'color' : 'tab:orange', 'linewidth': 3},
                 boxprops={'color' : 'tab:orange'},
                 flierprops={'markersize':0.01}
                )
res = {key : [v.get_ydata() for v in value] for key, value in bp.items()}
ax.plot(xticks, np.array(res['medians'])[:,0], 'x-', color='tab:orange',
        label='LSTM', alpha=alpha, linewidth=linewidth)
for b, c in zip(bp['boxes'], cols):
    b.set_alpha(alpha)


xticks = np.concatenate((np.array(num_grids_non_adap_list2), np.array(num_grids_non_adap_list1)))
plt.xticks(xticks,
           list(map(str, xticks)))
plt.ylim([0, 0.4])
plt.xlabel('number of grids', fontsize=14)
plt.ylabel('MSE', fontsize=14)
plt.legend(fontsize=14)
plt.grid()
plt.suptitle('Spiral data: Testing Precition MSE vs. Complexity (boxplot)', fontsize=16)
# plt.savefig('spiral_error_boxplot.eps', format='eps', dpi=1000)

plt.show()

